In [2]:
import numpy as np
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import optimizers
import time
#from sklearn.cross_validation import train_test_split
import pylab
import matplotlib.pyplot as plt
import pickle
import csv
#import pandas as pd

In [3]:
gpu_flag = 0

if gpu_flag >= 0:
    cuda.check_cuda_available()
xp = cuda.cupy if gpu_flag >= 0 else np

batchsize = 2000
n_epoch = 1


with open('/home/dl-box/Liver/sotsuron_train/train_pkl_data/new0921mix_NC.pkl','rb') as f1:
    dataset1 = pickle.load(f1)
    
with open('/data1/20171127/Test_dataset/FNH/NC/FNH208_NC.pkl','rb') as f2:
    dataset2 = pickle.load(f2)
    
train_set_x, train_set_y = dataset1[0]
#validation_set_x,validation_set_y = dataset1[1]
test_set_x, test_set_y = dataset2[0]

train_set_x = np.array(train_set_x).astype(xp.float32)
train_set_y = np.array(train_set_y).astype(xp.int32)
#validation_set_x = np.array(validation_set_x).astype(xp.float32)
#validation_set_y = np.array(validation_set_y).astype(xp.float32)
test_set_x = np.array(test_set_x).astype(xp.float32)
test_set_y = np.array(test_set_y).astype(xp.int32)

x_train = train_set_x.reshape(train_set_x.shape[0],-1)
y_train = train_set_y

#x_valid = validation_set_x.reshape(validation_set_x.shape[0],-1)
#y_valid = validation_set_y

x_test = test_set_x.reshape(test_set_x.shape[0],-1)
y_test = test_set_y


N = x_train.shape[0]

N_test = y_test.size

x_train = x_train.reshape((len(x_train), 1, 29, 29))
x_test = x_test.reshape((len(x_test), 1, 29, 29))

class LeNet(chainer.Chain):
    def __init__(self):
        super(LeNet,self).__init__(
        conv1=F.Convolution2D(1, 32, 3),   
        conv2=F.Convolution2D(32, 64, 3), 
        conv3=F.Convolution2D(64, 64, 2),
        conv4=F.Convolution2D(64, 128, 2),
        conv5=F.Convolution2D(128, 128, 2),
        conv6=F.Convolution2D(128, 128, 2),
        conv7=F.Convolution2D(128, 256, 2),
        conv8=F.Convolution2D(256, 256, 2),
        conv9=F.Convolution2D(256, 256, 2),
        conv10=F.Convolution2D(256, 512, 2),
        conv11=F.Convolution2D(512, 512, 2),
        conv12=F.Convolution2D(512, 512, 2),
            l1=F.Linear(2048, 128),            
            l2=F.Linear(128, 2)
        )
        
    def __call__(self,x_data,train = True):
        #x = chainer.Variable(x_data)
        x = chainer.Variable(x_data.astype(np.float32))
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = F.relu(self.conv5(h))
        h = F.relu(self.conv6(h))
        h = F.relu(self.conv7(h))
        h = F.relu(self.conv8(h))
        h = F.relu(self.conv9(h))
        h = F.relu(self.conv10(h))
        h = F.relu(self.conv11(h))
        h = F.relu(self.conv12(h))
        h = F.dropout(F.relu(self.l1(h)), train= train)
        y = self.l2(h)
        y = F.relu(y)
        return y

    def forward_Train(self, x_data,y_data):
        y = self(x_data ,True)
        t = chainer.Variable(y_data)
        return F.softmax_cross_entropy(y, t), F.accuracy(y,t)
    
    
    def forward_Test(self, x_data, y_data):
        t = chainer.Variable(y_data)
        y = self(x_data)
        return F.accuracy(y, t)
    
        
    def forward_label(self, x_data,y_data):
        label_list = []
        t = chainer.Variable(y_data)
        y = self(x_data)
        #print y.data
        
        for i in range(0,y.data.shape[0]):
            if y.data[i][0] > y.data[i][1]:
                label = 0
            else:
                label = 1 

            label_list = np.append(label_list,label) 
        """
           
        #↓for FNH207
        for i in range(0,y.data.shape[0]):
            if y.data[i][0] > y.data[i][1]:
                label = 1
            else:
                label = 0 

            label_list = np.append(label_list,label) 

        """

        return label_list




model = LeNet()
optimizer = optimizers.Adam()
optimizer.setup(model)

if gpu_flag >= 0:
    cuda.get_device(gpu_flag).use()
    model.to_gpu()

f = open('/home/dl-box/Liver/sotsuron_data/chainer_FNH208NC_12_0927_bs2000_l1_128.csv','w')
Write = csv.writer(f, lineterminator='\n')
# 訓練ループ
start_time = time.clock()
min_result = 1.000

acc_label2 = []

for epoch in range(1, n_epoch + 1):
    print "epoch: %d" % epoch

    perm = np.random.permutation(N)
    sum_loss = 0
    train_accuracy = 0
    for i in range(0, N, batchsize):
        x_batch = xp.asarray(x_train[perm[i:i + batchsize]],dtype=np.float32)
        y_batch = xp.asarray(y_train[perm[i:i + batchsize]],dtype=np.int32)
        #y_batch = xp.asarray(y_train[perm[i:i + batchsize]],dtype=np.int32)
        
        model.zerograds()
        loss,train_acc = model.forward_Train(x_batch, y_batch)
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(y_batch)
        train_accuracy += float(train_acc.data) * len(y_batch)
        
        train_result = train_accuracy / N
        result = sum_loss / N

    print "train mean loss: %f" %(result)
    print "train accuracy: %f" %(train_result)
    
    

epoch: 1
train mean loss: 0.693769
train accuracy: 0.499957


In [13]:
print len(train_set_x)
print len(train_set_y)
print len(test_set_x)
print len(test_set_y)

#print train_set_x[0]
#print test_set_x[0]
#print train_set_y
#print test_set_y
print x_train[0][0][0]

393760
393760
226305
226305
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.125       0.203125    0.11328125
  0.0390625   0.04296875  0.06640625  0.05078125  0.02734375  0.046875
  0.04296875  0.03125     0.01171875]
